In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from datetime import date
today = date.today()

import undetected_chromedriver as uc

import time
import pandas as pd

In [2]:

############################################################################

driver = uc.Chrome()

###########################################################################

driver.get("https://app.prizepicks.com/")
time.sleep(15)
elem = driver.find_element(By.CLASS_NAME, "close")
ac = ActionChains(driver)
ac.move_to_element(elem).move_by_offset(0, 0).click().perform()
time.sleep(3)

ppPlayers = []

# CHANGE NBA TO ANY SPORT THAT YOU LIKE!!!!! IF THE SPORT IS NOT OFFERED ON PP THEN THE PROGRAM WILL RUN AN ERROR AND EXIT.
driver.find_element(By.XPATH, "//span[@class='name'][normalize-space()='NBASL']").click()
time.sleep(3)

stat_container = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.CLASS_NAME, "stat-container")))

categories = driver.find_element(By.CSS_SELECTOR, ".stat-container").text.split('\n')
for category in categories[1:]:
    test = category.split(' ')
    yeayuh = "(Combo)"
    if yeayuh not in test: 
        driver.find_element(By.XPATH, f"//div[text()='{category}']").click()

        projectionsPP = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#test-projection-li")))
        for projections in projectionsPP:
            names = projections.find_element(By.CSS_SELECTOR, "#test-player-name").text
            prop = projections.find_element(By.CLASS_NAME, "self-center").text
            split = prop.split("\n")
            team = projections.find_element(By.CSS_SELECTOR, "#test-team-position").text
            split_1 = team.split(" ")
            vteam = projections.find_element(By.CSS_SELECTOR, "[aria-label='Start Time']").text
            split_2 = vteam.split(" ")
            try:
                type = vteam = projections.find_element(By.CSS_SELECTOR, "[aria-label='Open modal for Demons and Goblins'] > img").get_attribute("alt")
            except:
                type = "Standard"
            
            players = {
                'Name': names,
                'Value': split[0],
                'Prop': split[-1],
                'Team': split_1[0],
                'vsTeam': split_2[1],
                'Type': type
            }
            ppPlayers.append(players)

dfProps = pd.DataFrame(ppPlayers)
# CHANGE THE NAME OF THE FILE TO YOUR LIKING

dfProps.to_csv(f'PPLines-{today}.csv')

print("These are all of the props offered by PP.", '\n')
print(dfProps)
print('\n')

These are all of the props offered by PP. 

                  Name Value           Prop Team vsTeam      Type
0         Dillon Jones   4.5        Assists  OKC    GSW  Standard
1   Zaccharie Risacher   5.0       Rebounds  ATL    CHI  Standard
2      Harrison Ingram   6.0       Rebounds  SAS    PHI  Standard
3          Kel’el Ware  10.0       Rebounds  MIA    TOR  Standard
4      Donovan Clingan  11.5       Rebounds  POR    CHA  Standard
5       Nick Smith Jr.   5.0       Rebounds  CHA    POR  Standard
6    Baylor Scheierman  23.5  Pts+Rebs+Asts  BOS    DAL  Standard
7          A.J. Lawson  25.0  Pts+Rebs+Asts  DAL    BOS  Standard
8        Matas Buzelis  25.5  Pts+Rebs+Asts  CHI    ATL  Standard
9   Zaccharie Risacher  22.0  Pts+Rebs+Asts  ATL    CHI  Standard
10        Adama Sanogo  25.5  Pts+Rebs+Asts  CHI    ATL  Standard
11         Ron Holland  27.0  Pts+Rebs+Asts  DET    NYK  Standard
12    Duane Washington  23.5  Pts+Rebs+Asts  NYK    DET  Standard
13        Jared McCain  24.0  Pt

In [3]:
# for further analysis of nba players

from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import teamvsplayer
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv(f'PPLines-{today}.csv')
del df['Unnamed: 0']
names = df["Name"]
team_abrev = df["Team"]
vteam_abrev = df["vsTeam"]
player_ids = []
team_ids = []
vteam_ids = []
nba_players = players.get_players()
nba_teams = teams.get_teams()

# collect player and team ids
for i in team_abrev:
    big_fundamental = [
        team for team in nba_teams if team['abbreviation'] == i
    ][0]
    team_ids.append(str(big_fundamental.get('id')))
for i in vteam_abrev:
    big_fundamental = [
        team for team in nba_teams if team['abbreviation'] == i
    ][0]
    vteam_ids.append(str(big_fundamental.get('id')))
#for i in names:
    #big_fundamental = [
        #player for player in nba_players if player["full_name"] == i
    #][0]
    #player_ids.append(str(big_fundamental.get('id')))
    #career = playercareerstats.PlayerCareerStats(player_id=big_fundamental.get('id'))
print(df)

                  Name  Value           Prop Team vsTeam      Type
0         Dillon Jones    4.5        Assists  OKC    GSW  Standard
1   Zaccharie Risacher    5.0       Rebounds  ATL    CHI  Standard
2      Harrison Ingram    6.0       Rebounds  SAS    PHI  Standard
3          Kel’el Ware   10.0       Rebounds  MIA    TOR  Standard
4      Donovan Clingan   11.5       Rebounds  POR    CHA  Standard
5       Nick Smith Jr.    5.0       Rebounds  CHA    POR  Standard
6    Baylor Scheierman   23.5  Pts+Rebs+Asts  BOS    DAL  Standard
7          A.J. Lawson   25.0  Pts+Rebs+Asts  DAL    BOS  Standard
8        Matas Buzelis   25.5  Pts+Rebs+Asts  CHI    ATL  Standard
9   Zaccharie Risacher   22.0  Pts+Rebs+Asts  ATL    CHI  Standard
10        Adama Sanogo   25.5  Pts+Rebs+Asts  CHI    ATL  Standard
11         Ron Holland   27.0  Pts+Rebs+Asts  DET    NYK  Standard
12    Duane Washington   23.5  Pts+Rebs+Asts  NYK    DET  Standard
13        Jared McCain   24.0  Pts+Rebs+Asts  PHI    SAS  Stan

In [4]:
# collect stats in ______ season for players
pd.options.mode.chained_assignment = None  # default='warn'
df["Diff"] = ""
df["Prop_avg"] = ""
for i in range(len(player_ids)):
    playoff_stats = playergamelog.PlayerGameLog(player_id=player_ids[i],season_type_all_star='Regular')
    #playoff_stats = playergamelog.PlayerGameLog(player_id=player_ids[i])
    playoff_stats_dfs = playoff_stats.get_data_frames()
    playoff_stats_df = playoff_stats_dfs[0]
    testerino = playoff_stats_df[["FG3M","FG3A", "FTM", "FTA", "REB", "AST", "STL", "BLK", "TOV", "PTS", "PLUS_MINUS"]].mean()
    
    propval = df['Value'][i]
    proptyp = df['Type'][i]
    prop = str(df['Prop'][i])
    match prop:
        case "Points":
            diff = abs(propval - testerino["PTS"])
            df["Prop_avg"][i] = testerino["PTS"]
        case "Rebounds":
            diff = abs(propval - testerino["REB"])
            df["Prop_avg"][i] = testerino["REB"]
        case "Assists":
            diff = abs(propval - testerino["AST"])
            df["Prop_avg"][i] = testerino["AST"]
        case "3-PT Made":
            diff = abs(propval - testerino["FG3M"])
            df["Prop_avg"][i] = testerino["FG3M"]
        case "Reb+Asts":
            diff = abs(propval - (testerino["REB"] + testerino["AST"]))
            df["Prop_avg"][i] = testerino["REB"] + testerino["AST"]
    df["Diff"][i] = diff

print(df.sort_values(by='Diff'))

                  Name  Value           Prop Team vsTeam      Type Diff  \
0         Dillon Jones    4.5        Assists  OKC    GSW  Standard        
20     Donovan Clingan   23.0  Pts+Rebs+Asts  POR    CHA  Standard        
19       Ajay Mitchell   23.0  Pts+Rebs+Asts  OKC    GSW  Standard        
18        Dillon Jones   25.0  Pts+Rebs+Asts  OKC    GSW  Standard        
17         Kel’el Ware   28.5  Pts+Rebs+Asts  MIA    TOR  Standard        
16           Adem Bona   15.5  Pts+Rebs+Asts  PHI    SAS  Standard        
15    Ricky Council IV   26.5  Pts+Rebs+Asts  PHI    SAS  Standard        
14      Jamaree Bouyea   19.0  Pts+Rebs+Asts  SAS    PHI  Standard        
13        Jared McCain   24.0  Pts+Rebs+Asts  PHI    SAS  Standard        
12    Duane Washington   23.5  Pts+Rebs+Asts  NYK    DET  Standard        
21      Nick Smith Jr.   27.5  Pts+Rebs+Asts  CHA    POR  Standard        
11         Ron Holland   27.0  Pts+Rebs+Asts  DET    NYK  Standard        
9   Zaccharie Risacher   

In [5]:
#performance against specific team?? // THIS SECTION OF THE API IS EITHER NOT WROKING OR I DONT UNDERSTAND IT
print(names[0])
teamvplayer_reg_season = teamvsplayer.TeamVsPlayer(per_mode_detailed='PerGame', vs_player_id=player_ids[0], team_id = team_ids[0], season_type_playoffs = 'Playoffs', opponent_team_id = vteam_ids[0])
ruroh = teamvplayer_reg_season.get_data_frames()[0]
gamesplayed = ruroh["GP"].iloc[0]
playoff_stats_real = ruroh[["FG3M","FG3A", "FTM", "FTA", "REB", "AST", "STL", "BLK", "TOV", "PTS", "PLUS_MINUS"]].iloc[0]/gamesplayed
print(teamvplayer_reg_season.get_data_frames())

Dillon Jones


IndexError: list index out of range

In [ ]:
import argparse

import requests


# Obtain the api key that was passed in from the command line
parser = argparse.ArgumentParser(description='Sample V4')
parser.add_argument('455ca26b0752882f9256c1ac332035ec', type=str, default='')
args = parser.parse_args()


# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = '455ca26b0752882f9256c1ac332035ec'

# Sport key
# Find sport keys from the /sports endpoint below, or from https://the-odds-api.com/sports-odds-data/sports-apis.html
# Alternatively use 'upcoming' to see the next 8 games across all sports
SPORT = 'basketball_nba'

# Bookmaker regions
# uk | us | us2 | eu | au. Multiple can be specified if comma delimited.
# More info at https://the-odds-api.com/sports-odds-data/bookmaker-apis.html
REGIONS = 'us'

# Odds markets
# h2h | spreads | totals. Multiple can be specified if comma delimited
# More info at https://the-odds-api.com/sports-odds-data/betting-markets.html
# Note only featured markets (h2h, spreads, totals) are available with the odds endpoint.
MARKETS = 'h2h,spreads'

# Odds format
# decimal | american
ODDS_FORMAT = 'decimal'

# Date format
# iso | unix
DATE_FORMAT = 'iso'

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds', params={
    'api_key': API_KEY,
    'regions': REGIONS,
    'markets': MARKETS,
    'oddsFormat': ODDS_FORMAT,
    'dateFormat': DATE_FORMAT,
})

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])